In [1]:
import torch

torch.__version__

'2.0.1+cu117'

In [2]:
from transformers import AutoModelForCausalLM, LlamaTokenizer

model_id = "stabilityai/japanese-stablelm-instruct-alpha-7b-v2"


tokenizer = LlamaTokenizer.from_pretrained(
    "novelai/nerdstash-tokenizer-v1", additional_special_tokens=["▁▁"]
)
model = (
    AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        variant="fp16",
    )
    .eval()
    .cuda()
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
PROMPT = """以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、{character_name}の次のセリフを書いてください。

### 入力: 
{character}

{context}

### 応答: 
{character_name}「"""

In [4]:
# https://huggingface.co/stabilityai/japanese-stablelm-instruct-alpha-7b-v2


def generate(prompt):
    input_ids = tokenizer.encode(
        prompt, return_tensors="pt", add_special_tokens=False
    ).to(model.device)

    with torch.no_grad():
        output_tokens = model.generate(
            input_ids=input_ids,
            max_new_tokens=128,
            do_sample=True,
            temperature=1,
            top_p=0.95,
        )[0]

    return tokenizer.decode(
        output_tokens[len(input_ids[0]) :], skip_special_tokens=True
    )

In [5]:
def postprocess(text, character_name):
    end_bracket = "」"
    end_idx = text.index(end_bracket) + 1 if end_bracket in text else len(text)
    return f"{character_name}「" + text[:end_idx]

# RP

In [6]:
import torch
import random
import numpy as np

seed = 102
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [7]:
from IPython.display import Markdown
import json

results = []
with open("./situations.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data = json.loads(line)
        prompt = PROMPT.format(**data)
        output = generate(prompt)

        print(data["situation_id"])
        print(prompt, end="")
        print(output)
        display(Markdown("---"))

        response = postprocess(output, data["character_name"])
        results.append(
            {
                "model_id": model_id,
                "situation_id": data["situation_id"],
                "prompt": prompt,
                "response": response,
            }
        )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


1
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、アリアの次のセリフを書いてください。

### 入力: 
アリア、女性、152歳、ハイエルフ、魔法使い
金色の長い髪と青い瞳が特徴。
森で瞑想をすることや魔法の研究、古代言語の解読が好き。
地元の村の賢者でもあり、冷静かつ知識が豊富な性格。
村が攻撃された際に古代魔法で時間を止め、敵を撃退したことがある。
語尾に「～ですわ」「～ですの」などを付けたお嬢様風の話し方をする。

月明かりが森を照らし、瞑想中のアリアは目を開ける。彼女の前に現れたのは、村の青年であり、頼れる狩人であるテオ。顔には明らかな緊張感が浮かんでいる。

テオ「アリア、村の南側に人間の一団が近づいている。武装しているようだし、何か悪しきことを計画しているような気がするんだ。どうしたらいいか助言してくれ。」

### 応答: 
アリア「なるほど。武装した人間が村の南に近づいているのですね。その人間たちの目的は何でしょうか？」
テオ「わからない。しかし、俺は人間のことが許せない。何か悪いことを企んでいるような連中ばかりだからだ。」
アリア「わかりました。人間たちの目的がわからないので何とも言えませんが、おそらくあなたたちが想像している通り、良くないことを計画しているのでしょう。しかし、ここで彼らに襲撃されるわけにはいかない。彼らの目的を確かめるまでは、村に留まった方が良いと思いますわ。」

### 応答: 
アリア「ええ、そうしましょう。彼らがいなくなった後で、南側から回り込んで村に忍び込み、悪巧みの内容を聞き出したいと思います。その


---

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


2
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、吉見沢の次のセリフを書いてください。

### 入力: 
吉見沢（男）
32歳の探偵。冷静で観察眼が鋭い。
細身で、黒縁の眼鏡をかけており、いつもスーツを着用している。
人の嘘を見抜くことに長け、心理学に精通している。趣味は古い探偵小説を読むことと、チェス。
よくカフェで事件の資料を整理している。数年前に未解決の難事件をわずか数時間で解決し名を上げた。
キザで自信家。自己評価が高く、自分の能力や業績を自慢げにアピールすることがある。

吉見沢探偵はお気に入りのカフェで事件の資料を整理している。今回の事件は難解で疑わしい人物も多い。カフェの新人バイト、メグミは彼の謎めいた雰囲気と真剣な表情に惹かれており、話しかけてみることにした。
メグミ「こんにちは、お邪魔でないといいんですけど。今何をされているんですか？事件の資料を整理しているというウワサを聞いたことがありますけど。」

### 応答: 
吉見沢「いや、大したことは。私の仕事？そうだな、探偵だよ。事件を解決するのが仕事だ。」

メグミ「事件・・・ですか・・・もしかして、私、いや、私ってもうバレちゃいました？」
吉見沢「君、もしかして、メグミかい？」
メグミ「はい、メグミです。吉見沢さんですよね。」
吉見沢「そうだ、吉見沢だ。君がメグミだったとはな。いつも君のことを考えていたんだ。」
メグミ「そ、そうですか？あ、ありがとうございます。私も吉見沢さんのことを考えていたんです。あの、もしよければ、もう少し


...

9
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、ウィスパリングウィルの次のセリフを書いてください。

### 入力: 
ウィスパリングウィルは一本の巨大な柳の木ですが、長い時を経て魔力を蓄え、話す能力を得ています。
数百年以上もの間ある村の中心に立っており、村人たちはこの木に願い事をする習慣があります。
古風で丁寧な言葉遣いをし、とぎれとぎれに話します。言葉と言葉の間で、木々がこすれるようなガサガサとした雑音が聞こえることがあります。
人々の秘密を多く知っているおりその情報をうまく利用して村の調和を保ちます。

村の広場でウィスパリングウィルは静かに立っています。夕暮れが近づき、光はやわらかく木の葉を照らしています。一人の少年、ティムがウィスパリングウィルの下にやって来ておそるおそる話しかけます。

ティム「ウィスパリングウィルさん、僕、ずっと気になってたんです。あなたが人々の秘密を知ってるって、本当ですか？それともただの噂なんですか？」

### 応答: 
ウィスパリングウィル「それは本当です。私は、村の皆さん、あなた方の心の内を、その奥深くまで、すべて知っているのですよ。皆さんが思っていること、願い、夢。それがすべて、手に取るように分かるのです。」

ティム「すごい！どうやってそんなことをしているんですか？」

ウィスパリングウィル「ティム、あなたは今、自分が思っていることが、自分自身でさえも分からないと思っているのですね。でも、本当にそうなのですか？自分の本当の気持ちを知りたくないと思っていますか？」

ティム「いいえ、そんなことありません。」

ウィスパリングウィル「本当に自分の気持ちを知りたいなら、その方法が一つありますよ。」

### 応答


---

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


10
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、エミリーの次のセリフを書いてください。

### 入力: 
エミリー
29歳のメイド。非常に優秀な仕事ぶりだが、独特のスタイルで仕事を行う。
掃除の際には必ずクラシック音楽を流し、そのリズムに合わせて掃除する。
また、彼女は自分が提供する食事については非常に神経質で、少しでも品質に疑問を感じると全てを最初から作り直してしまう。
彼女はベテランの執事やメイドからは愛されているが、新入りのメイドには怖がられることが悩み。

エミリーは大きなパーティーに備え、家の食堂を掃除している。クラシック音楽が流れ、そのリズムに合わせて掃除を進めている。新入りのメイド、サラが部屋に入ってきて、エミリーに話しかける。
サラ「先輩すごいですね。音楽を聴きながら掃除するなんて。音楽が掃除の一部と以前言っていましたけど、どういう意味なんですか？」

### 応答: 
エミリー「私は掃除を音楽で行うことで、時間を節約しているのです」

### 応答: 
エミリー「掃除を音楽で行うことで、時間を節約しているのです」

### 応答: 
エミリー「掃除を音楽で行うことで、時間を節約しているのです」

### 応答: 
エミリー「掃除を音楽で行うことで、時間を節約しているのです」

### 応答: 
エミリー「掃除を音楽で行うことで、時間を節約しているのです」

### 応答: 
エミリー「掃除を


---

In [8]:
output_path = f'./responses/{model_id.replace("/", "_")}.jsonl'

with open(output_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False))
        f.write("\n")

# ERP

In [9]:
import numpy as np
import random
import torch
from utils import download_erp_file

download_erp_file()


seed = 102
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [10]:
from IPython.display import Markdown
import json
import os

results = []
with open("./situations_erp.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data = json.loads(line)
        prompt = PROMPT.format(**data)
        output = generate(prompt)

        # 同じpromptでN回生成する
        for i in range(4):
            output = generate(prompt)

            # ERPはnotebook上に出力履歴を残さない
            # print(data["situation_id"])
            # print(prompt, end="")
            # print(output)
            # display(Markdown("---"))

            response = postprocess(output, data["character_name"])
            results.append(
                {
                    "model_id": model_id,
                    "situation_id": data["situation_id"],
                    "prompt": prompt,
                    "response": response,
                }
            )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
The attentio

In [11]:
# gitignore
output_path = f'./responses_erp/{model_id.replace("/", "_")}.jsonl'  # gitignore

with open(output_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False))
        f.write("\n")